In [29]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
import copy
import torch
from utils import poly_gon_and_line,bboxes_overlapping,bboxes_of_poly
from data_loader import Waymo_Motion_DataLoader

config = dict()
config['train'] = '/home/avt/prediction/Waymo/dataset/train'
config['val'] = '/home/avt/prediction/Waymo/dataset/validation'
config['pred_range'] = [-100.0, 100.0, -100.0, 100.0]
config['num_scales'] = 6
config['cross_dist'] = 6


class Waymo_Motion_Preprocess_4:
    
    def __init__(self, scenario_list: list, config) -> None:
        """Initialization function for the class.

        Args:
            scenario_list: A list of scenarios.
            config: A dict for configuration.

        Note:
            scenario_list is directly parsed from the TFRecord by Waymo_Motion_Dataset_Loader.read_TFRecord.
        """

        self.counter: int = 0

        self.config: dict = config
        self.scenario_list: list = scenario_list
        self.current_scenario: dict = scenario_list[self.counter]

    
    def __getitem__(self, index: int) -> dict:
        """
        Args:
            index
        
        Notes:
            A scenario is a dict of 9 keys:

            'currentTimeIndex',
            'dynamicMapStates'
            'mapFeatures',
            'objectsOfInterest', 
            'scenarioId', 
            'sdcTrackIndex',
            'timestampsSeconds', 
            'tracks', 
            'tracksToPredict'

            get_road_info() and get_graph() are deployed for 'dynamicMapStates' and 'mapFeatures'
            
            get_obj_states() and get_obj_feats() are for the rest.

        """

        data = self.get_obj_states(index)
        data = self.get_obj_feats(data)

        data['road_info'] = self.get_road_info(data)
        data['graph'] = self.get_graph(data)
        #data['graph'] = self.get_left_right(data['graph'])
        
        return data
    

    def __len__(self) -> int:
        """Get the number of scenarios in the list.

        Returns:
            Number of scenarios.
        """

        return len(self.scenario_list)
    

    def get_obj_states(self, index: int) -> dict:
        """Get the states of objects in a scenario corresponding to the given index.

        Args:
            index: index of scenario
        
        Returns:
            data: a dict with dict_keys(['index', 
                                        'objects_of_interest', 
                                        'time_stamps', 
                                        'current_time_index', 
                                        'scenario_id', 
                                        'sdc_index', 
                                        'trajs_xyz', 
                                        'valid_masks', 
                                        'velocity_xy_heading', 
                                        'shapes', 
                                        'object_ids', 
                                        'object_types', 
                                        'track_to_predict_index', 
                                        'track_to_predict_ids'])
        
        Notes:
            The 'objects_of_interest' is missing in a fair amount of scenarios, in which case dict['objects_of_interest'] = None.
            
            'sdc_index': index of Self-Driving Car.

        """

        scen = self.scenario_list[index]

        data = dict()

        if 'objectsOfInterest' in scen.keys():
            data['objects_of_interest'] = scen['objectsOfInterest']
        else:
            data['objects_of_interest'] = None

        data['time_stamps'] = np.array(scen['timestampsSeconds'])
        data['current_time_index'] = scen['currentTimeIndex']
        data['scenario_id'] = scen['scenarioId']
        data['sdc_index'] = scen['sdcTrackIndex']


        obj_id, obj_type, obj_valid,obj_traj,obj_velocity_heading,obj_shape = self.get_tracks_info(scen['tracks'])

        ttp_indx = [track['trackIndex'] for track in scen['tracksToPredict']]
        ttp_indx = np.array(ttp_indx)

        data['trajs_xyz'] = obj_traj
        data['valid_masks'] = obj_valid
        data['velocity_xy_heading'] = obj_velocity_heading
        data['shapes'] = obj_shape
        data['object_ids'] = obj_id
        data['object_types'] = obj_type
        data['track_to_predict_index'] = ttp_indx
        data['index'] = index

        return data
    

    def get_obj_feats(self, data: dict) -> dict:

        orig = data['trajs_xyz'][data['sdc_index']][data['current_time_index']]
        pre_orig = data['trajs_xyz'][data['sdc_index']][data['current_time_index']-1]
        
        dir_vec = pre_orig - orig
        
        theta = np.pi - np.arctan2(dir_vec[1], dir_vec[0])
        rot = np.asarray([
                    [np.cos(theta), -np.sin(theta)],
                    [np.sin(theta), np.cos(theta)]], np.float32)

        data['orig'] = orig
        data['theta'] = theta
        data['rot'] = rot

        feats, ctrs, gt_preds, has_preds, engage_id, engage_index = [], [], [], [], [], []
        gt2_preds = []
        
        for i in range(len(data['object_ids'])):

            feat = np.zeros((11, 4), np.float32)
            traj_xyz = data['trajs_xyz'][i][:11]

            mask_i = data['valid_masks'][i][:11] 

            if mask_i[-1] != True:
                continue
    
            reverse = list(np.flip(mask_i))
            if False in reverse:
                index = -reverse.index(False)
                
                traj_xyz = traj_xyz[index:,]

                # feat[index:, :2] = np.matmul(rot, (traj_xyz - orig)[:,:2].T).T
                # feat[index:,2] = (traj_xyz - orig)[:,2]
                feat[index:,:3] = rotation(traj_xyz, orig, theta)
                feat[index:,3] = 1.0

            else:
                index = 0
                # feat[:, :2] = np.matmul(rot, (traj_xyz - orig)[:,:2].T).T
                # feat[:,2] = (traj_xyz - orig)[:,2]
                feat[:,:3] = rotation(traj_xyz, orig, theta)
                feat[:,3] = 1.0

            #x_min, x_max, y_min, y_max = self.config['pred_range']
            # if feat[-1, 0] < x_min or feat[-1, 0] > x_max or feat[-1, 1] < y_min or feat[-1, 1] > y_max:
            #     continue

            mask_gt = np.arange(11,91)
            gt_pred = data['trajs_xyz'][i][mask_gt]

            has_pred = data['valid_masks'][i][mask_gt]

            # if np.linalg.norm(tmp[-1] - tmp[0])<1:
            #     continue

            ctrs.append(feat[-1, :3].copy())
            feat[1:, :3] -= feat[:-1, :3]
            feat[index, :3] = 0

            feats.append(feat) 
            engage_id.append(data['object_ids'][i])
            engage_index.append(i)
            gt_preds.append(gt_pred)
            has_preds.append(has_pred)

            gt_pred2 = rotation(gt_pred, orig, theta)
            gt2_preds.append(gt_pred2)

        data['feats'] = feats
        data['ctrs'] = ctrs
        data['gt_preds'] = gt_preds 
        data['has_preds'] = has_preds
        data['engage_id'] = engage_id
        data['engage_index'] = engage_index
        data['gt2_preds'] = gt2_preds


        return data
    

    def get_road_info(self, data: dict) -> dict:

        road_keys = dict()

        road_keys['driveway'] = ['polygon']
        road_keys['crosswalk'] = ['polygon']
        road_keys['speedBump'] = ['polygon']
        road_keys['stopSign'] = ['position', 'lane']
        road_keys['roadLine'] = ['polyline', 'type']
        road_keys['roadEdge'] = ['polyline', 'type']
        

        index = data['index']
        scen = self.scenario_list[index]
        map_feature = dict()

        for mf in scen['mapFeatures']:
            key = list(mf.keys())[1]
            if key in map_feature.keys():
                map_feature[key] += [mf]
            else:
                map_feature[key] = [mf]
        
        road_info = dict()
        for key in map_feature.keys():
            if key == 'lane':
                road_info[key] = road_info_lane(map_feature[key])     
            else:
                road_info[key] = road_info_except_lane(map_feature[key],road_keys)
        
        
        if 'roadEdge' in road_info.keys():
            copy_info = copy.deepcopy(road_info['roadEdge'])
            if 'roadLine' in road_info.keys():
                for key in copy_info.keys():
                    copy_info[key] += road_info['roadLine'][key]
            
        elif 'roadLine' in road_info.keys():
            copy_info = copy.deepcopy(road_info['roadLine'])
        
        else:
            copy_info = dict()
        
        road_info['road_Edge_and_Lines'] = copy_info


        dynamic = scen['dynamicMapStates']
        road_info['dynamic_map'] = get_dynamic_map(dynamic)
        
        return road_info
    

    def get_graph(self, data: dict) -> dict:

        engage_lanes = get_engage_lanes(data,self.config)
        edge_line_id = data['road_info']['road_Edge_and_Lines']['id']

        ctrs, feats,node_idcs,num_nodes = get_nodes(engage_lanes)
        
        pre_pairs,suc_pairs, left_pairs, right_pairs,left_boundaries,right_boundaries = get_lane_pairs(engage_lanes,edge_line_id)
        pre, suc, left, right, lane_idcs = get_node_pairs(engage_lanes,left_pairs,right_pairs)
       

        graph = dict()
        graph['ctrs'] = np.concatenate(ctrs, 0)
        graph['feats'] = np.concatenate(feats, 0)
        graph['num_nodes'] = num_nodes 
        graph['node_idcs'] = node_idcs

        graph['pre'] = [pre]
        graph['suc'] = [suc]
        graph['pre_pairs'] = pre_pairs   
        graph['suc_pairs'] = suc_pairs  

        graph['left_pairs'] = left_pairs  
        graph['right_pairs'] = right_pairs
        graph['left'] = left
        graph['right'] = right

        graph['letf_boundaries'] = left_boundaries
        graph['right_boundaries'] = right_boundaries

        graph['lane_idcs'] = lane_idcs   

        for k1 in ['pre', 'suc']:
            for k2 in ['u', 'v']:
                graph[k1][0][k2] = np.asarray(graph[k1][0][k2], np.int64)

        for key in ['pre', 'suc']:
            graph[key] += dilated_nbrs(graph[key][0], graph['num_nodes'], self.config['num_scales'])

        return graph


    def get_tracks_info(self, tracks: list) -> list:
        """Transform the 'tracks' into a reader-friendly format. 
        
        Args:
            scenario_list[index]['tracks']
        
        Note:
            obj_velocity_heading: (v_x,v_y,heading)
        """
        
        obj_id, obj_type, obj_traj, obj_shape, obj_velocity_heading, obj_valid = [],[],[],[],[],[]
        
        for track in tracks:
            obj_id += [track['id']]
            obj_type += [track['objectType']]

            states = track['states']
            states_valid, states_traj, states_velocity_heading = [],[],[]
            
            for state in states:
                states_valid += [state['valid']]
                
                if state['valid']:
                    states_traj += [[state['centerX'],state['centerY'],state['centerZ']]]
                    states_velocity_heading += [[state['velocityX'],state['velocityY'],state['heading']]]
                else:
                    states_traj += [[0,0,0]]
                    states_velocity_heading += [[0,0,0]]
        
            if True in states_valid:
                valid_index = states_valid.index(True)
                valid_state = states[valid_index]
                states_shape = [valid_state['length'],valid_state['width'],valid_state['height']]
            else:
                states_shape = None
            
            obj_valid += [np.array(states_valid)]
            obj_traj += [np.array(states_traj)]
            obj_velocity_heading += [np.array(states_velocity_heading)]
            obj_shape += [np.array(states_shape)]

        return obj_id, obj_type, obj_valid,obj_traj,obj_velocity_heading,obj_shape


def road_info_except_lane(x_list, road_keys):

    output = {}
    output['id'] = []
    
    key_x = list(x_list[0].keys())[1]
    keys = road_keys[key_x]
    for key in keys:
        output[key] = []

    for x in x_list:
        output['id'] += [x['id']]
        for key in keys:
            if key in list(x[key_x].keys()):
                if key[0] == 'p':
                    output[key] += [poly_gon_and_line(x[key_x][key])]
                else:
                    output[key] += [x[key_x][key]]
            else:
                output[key] += [None]
    
    return output


def road_info_lane(x_dict):
    
    lanes = dict()

    for ln in x_dict:
        
        ln_info = dict()
        ln_id = ln['id']

        for key in ln['lane'].keys():
            if key[0] == 'p':
                ln_info[key] = poly_gon_and_line(ln['lane']['polyline'])
            else:
                ln_info[key] = ln['lane'][key]

        lanes[ln_id] = ln_info
    
    return lanes


def dilated_nbrs(nbr, num_nodes, num_scales):
    """
    Given the  adjacent matrix of distance=1,
    calculate the adjacent matrix of distance=2^i, i in range(1,num_scales)

    Args:
        nbr: dict(),
        graph['pre'] or graph['suc]

        num_nodes:  int, graph['num_nodes']

        num_scales:  int, config["num_scales"]

    Returns: a dict of adjacent matrix in coordinate form

    """

    data = np.ones(len(nbr['u']), np.bool)
    csr = sparse.csr_matrix((data, (nbr['u'], nbr['v'])), shape=(num_nodes, num_nodes))
    mat = csr
    nbrs = []
    
    for i in range(1, num_scales):
        mat = mat * mat
        nbr = dict()
        coo = mat.tocoo()   # converts a sparse matrix to coordinate format
        nbr['u'] = coo.row.astype(np.int64)
        nbr['v'] = coo.col.astype(np.int64)
        nbrs.append(nbr)

    return nbrs


def get_nodes(engage_lanes):

    ctrs, feats = [], []
    lane_ids = list(engage_lanes.keys())

    for id in lane_ids:
        lane = engage_lanes[id]
        ctrln = lane['polyline']
        ctrs.append(np.asarray((ctrln[:-1] + ctrln[1:]) / 2.0, np.float32))
        feats.append(np.asarray(ctrln[1:] - ctrln[:-1], np.float32))

    node_idcs = []
    count = 0
    for i, ctr in enumerate(ctrs):
        node_idcs.append(range(count, count + len(ctr)))
        count += len(ctr)
    num_nodes = count
    
    return ctrs, feats, node_idcs, num_nodes


def get_lane_pairs(engage_lanes,edge_line_id):

    lane_ids = list(engage_lanes.keys())

    pre_pairs, suc_pairs, left_pairs, right_pairs = [], [], [], []
    left_boundaries, right_boundaries = [],[]

    for i, lane_id in enumerate(lane_ids):

        lane = engage_lanes[lane_id]

        if 'entryLanes' in lane.keys():
            for eL in lane['entryLanes']:
                if eL in lane_ids:
                    j = lane_ids.index(eL)
                    pre_pairs += [[i,j]]
        
        if 'exitLanes' in lane.keys():
            for eL in lane['exitLanes']:
                if eL in lane_ids:
                    j = lane_ids.index(eL)
                    suc_pairs += [[i,j]]
        
        if 'leftNeighbors' in lane.keys():
            neighbors = lane['leftNeighbors']
            for nn in neighbors:
                n_id = nn['featureId']

                indx_s1 = nn['selfStartIndex']
                indx_s2 = nn['selfEndIndex']
                indx_n1 = nn['neighborStartIndex']
                indx_n2 = nn['neighborEndIndex']

                j = lane_ids.index(n_id)
                pair = [i, j, indx_s1, indx_s2, indx_n1, indx_n2]
                left_pairs.append(pair)
        
        if 'rightNeighbors' in lane.keys():
            neighbors = lane['rightNeighbors']
            for nn in neighbors:
                n_id = nn['featureId']
                
                indx_s1 = nn['selfStartIndex']
                indx_s2 = nn['selfEndIndex']
                indx_n1 = nn['neighborStartIndex']
                indx_n2 = nn['neighborEndIndex']

                j = lane_ids.index(n_id)
                pair = [i, j, indx_s1, indx_s2, indx_n1, indx_n2]
                right_pairs.append(pair)


        if 'leftBoundaries' in lane.keys():
            for eL in lane['leftBoundaries']:
                
                if eL['boundaryFeatureId'] in edge_line_id:
                    j = edge_line_id.index(eL['boundaryFeatureId'])
                else:
                    j = None

                left_boundaries += [[i,j]]
        
        if 'rightBoundaries' in lane.keys():
            for eL in lane['rightBoundaries']:
                
                if eL['boundaryFeatureId'] in edge_line_id:
                    j = edge_line_id.index(eL['boundaryFeatureId'])
                else:
                    j = None

                right_boundaries += [[i,j]]    

    pre_pairs = np.asarray(pre_pairs, np.int64)
    suc_pairs = np.asarray(suc_pairs, np.int64)
    left_pairs = np.asarray(left_pairs, np.int64)
    right_pairs = np.asarray(right_pairs, np.int64)   
    left_boundaries = np.asarray(left_boundaries, np.int64) 
    right_boundaries = np.asarray(right_boundaries, np.int64)  

    return pre_pairs,suc_pairs,left_pairs,right_pairs,left_boundaries,right_boundaries


def get_node_pairs(engage_lanes, left_pairs, right_pairs):
    lane_ids = list(engage_lanes.keys())

    ctrs, feats,node_idcs,num_nodes = get_nodes(engage_lanes)
    
    #---------------------------pre,suc-------------------------------#
    pre, suc = dict(), dict()
    
    for key in ['u', 'v']:
        pre[key], suc[key] = [], []
        
    for i, lane_id in enumerate(lane_ids):
        lane = engage_lanes[lane_id]
        idcs = node_idcs[i]

        pre['u'] += idcs[1:]
        pre['v'] += idcs[:-1]

        if 'entryLanes' in lane.keys():
            for eL in lane['entryLanes']:
                if eL in lane_ids:
                    j = lane_ids.index(eL)
                    pre['u'].append(idcs[0])
                    pre['v'].append(node_idcs[j][-1])

        suc['u'] += idcs[:-1]
        suc['v'] += idcs[1:]

        if 'exitLanes' in lane.keys():
            for eL in lane['exitLanes']:
                if eL in lane_ids:
                    j = lane_ids.index(eL)
                    suc['u'].append(idcs[-1])
                    suc['v'].append(node_idcs[j][0])
                
    #---------------------------left,right-------------------------------#
    left, right = dict(), dict()
    left['u'], left['v'] = [], []
    right['u'], right['v'] = [], []

    for p in left_pairs:

        ctr_s = torch.tensor(ctrs[p[0]][p[2]:p[3]])[:,:2]
        nodeId_s = np.array(node_idcs[p[0]][p[2]:p[3]])

        ctr_n = torch.tensor(ctrs[p[1]])[:,:2]
        nodeId_n = np.array(node_idcs[p[1]])
    
        dist = ctr_s.unsqueeze(1) - ctr_n.unsqueeze(0)
        dist = torch.sqrt((dist ** 2).sum(2))
        min_dist, min_idcs = dist.min(1)

        left['u'] += list(nodeId_s)
        left['v'] += list(nodeId_n[min_idcs.numpy()])
    
    for p in right_pairs:

        ctr_s = torch.tensor(ctrs[p[0]][p[2]:p[3]])[:,:2]
        nodeId_s = np.array(node_idcs[p[0]][p[2]:p[3]])

        ctr_n = torch.tensor(ctrs[p[1]])[:,:2]
        nodeId_n = np.array(node_idcs[p[1]])
    
        dist = ctr_s.unsqueeze(1) - ctr_n.unsqueeze(0)
        dist = torch.sqrt((dist ** 2).sum(2))
        min_dist, min_idcs = dist.min(1)

        right['u'] += list(nodeId_s)
        right['v'] += list(nodeId_n[min_idcs.numpy()])
    
    left['u'] = np.array(left['u'])
    left['v'] = np.array(left['v'])
    right['u'] = np.array(right['u'])
    right['v'] = np.array(right['v'])
    
    lane_idcs = []
    for i, idcs in enumerate(node_idcs):
        lane_idcs.append(i * np.ones(len(idcs), np.int64))

    lane_idcs = np.concatenate(lane_idcs, 0)
    
    return pre, suc, left, right, lane_idcs


def find_lane_ids_within_manhattan_range(lanes,origon,mht_range):
    """ 
    return the id of lanes within the manhattan range from the origon
    """
    mht_range = abs(mht_range)

    x_min = origon[0] - mht_range
    x_max = origon[0] + mht_range
    y_min = origon[1] - mht_range
    y_max = origon[1] + mht_range

    bbx_1 = [x_min,x_max,y_min,y_max]

    lane_id_list = []

    for key in lanes.keys():
        bbx_2 = bboxes_of_poly(lanes[key]['polyline'])
        if bboxes_overlapping(bbx_1,bbx_2):
            lane_id_list += [key]
    
    return lane_id_list


def get_engage_lanes(data,config):

    lanes = data['road_info']['lane']
    orig = data['orig']
    theta= data['theta']
    engage_lanes = dict()

    if 'manhattan' in config and config['manhattan']:
        x_min, x_max, y_min, y_max = config['pred_range']
        radius = max(abs(x_min), abs(x_max)) + max(abs(y_min), abs(y_max))
        lane_ids = find_lane_ids_within_manhattan_range(lanes,orig,radius)
        lane_ids = copy.deepcopy(lane_ids)
    
    else:
        lane_ids = list(lanes.keys())

    for id in lane_ids:
        lane = lanes[id]
        if len(lane['polyline']) < 2: #rule out those 1 point lane
            continue
        else:
            lane = copy.deepcopy(lane)
            # polyline_xy = np.matmul(rot, (lane['polyline'].T[:2].T- orig[:2].reshape(-1, 2)).T).T
            # polyline_xyz = np.vstack((polyline_xy.T,lane['polyline'].T[2])).T
            polyline_xyz = rotation(lane['polyline'],orig,theta)
            lane['polyline'] = polyline_xyz
            engage_lanes[id] = lane
            
    return engage_lanes


def get_dynamic_map(dynamic):

    output = dict()
    
    for i,dd in enumerate(dynamic):
        if 'laneStates' in dd.keys():
            lane_states = dd['laneStates']
            for Ls in lane_states:
                lane_id = Ls['lane']
                if lane_id in output.keys():
                    output[lane_id]['time_step'] += [i]
                    output[lane_id]['state'] += [Ls['state']]
                else:
                    output[lane_id] = dict()
                    output[lane_id]['stop_point_xyz'] = poly_gon_and_line(Ls['stopPoint'])
                    output[lane_id]['time_step'] =[i]
                    output[lane_id]['state'] = [Ls['state']]
        else:
            continue
    
    return output if output.keys() else None


def rotation(traj_xyz,orig,theta):

    rot = np.asarray([
                    [np.cos(theta), -np.sin(theta)],
                    [np.sin(theta), np.cos(theta)]], np.float32)
    
    traj_xy_r = np.matmul(rot, (traj_xyz - orig)[:,:2].T)
    traj_z = (traj_xyz  - orig)[:,2]
    traj_xyz_r = np.vstack((traj_xy_r,traj_z)).T

    return traj_xyz_r






In [2]:
train_dataset = Waymo_Motion_DataLoader(config['train'])

In [3]:
for tfr in train_dataset:
    print(tfr)

Path: /home/avt/prediction/Waymo/dataset/train
        ------------------------------
        || # TFRecord No.0 of total 10
        || # File: uncompressed_scenario_training_training.tfrecord-00003-of-01000
        ------------------------------
Path: /home/avt/prediction/Waymo/dataset/train
        ------------------------------
        || # TFRecord No.1 of total 10
        || # File: uncompressed_scenario_training_training.tfrecord-00001-of-01000
        ------------------------------
Path: /home/avt/prediction/Waymo/dataset/train
        ------------------------------
        || # TFRecord No.2 of total 10
        || # File: uncompressed_scenario_training_training.tfrecord-00008-of-01000
        ------------------------------
Path: /home/avt/prediction/Waymo/dataset/train
        ------------------------------
        || # TFRecord No.3 of total 10
        || # File: uncompressed_scenario_training_training.tfrecord-00002-of-01000
        ------------------------------
Path: /home/

In [4]:
parsed_list = train_dataset[2].read_TFRecord

2023-05-29 12:16:57.509478: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-29 12:16:57.509757: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2023-05-29 12:16:57.509804: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


TFRecord No.2 has been parsed successfully!
        ------------------------------
        || # Number of Scenarios: 497
        ------------------------------


In [15]:
processed_list = Waymo_Motion_Preprocess_4(parsed_list,config)

In [34]:
zz = processed_list[193]
zz.keys()

/tmp/ipykernel_572560/751831544.py:425: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.ones(len(nbr['u']), np.bool)


dict_keys(['objects_of_interest', 'time_stamps', 'current_time_index', 'scenario_id', 'sdc_index', 'trajs_xyz', 'valid_masks', 'velocity_xy_heading', 'shapes', 'object_ids', 'object_types', 'track_to_predict_index', 'index', 'orig', 'theta', 'rot', 'feats', 'ctrs', 'gt_preds', 'has_preds', 'engage_id', 'engage_index', 'gt2_preds', 'road_info', 'graph'])

In [37]:
zz['graph']['pre']

[{'u': array([   1,    2,    3, ..., 7984, 7985, 7986]),
  'v': array([   0,    1,    2, ..., 7983, 7984, 7985])},
 {'u': array([   2,    3,    4, ..., 7984, 7985, 7986]),
  'v': array([   0,    1,    2, ..., 7982, 7983, 7984])},
 {'u': array([   4,    5,    6, ..., 7984, 7985, 7986]),
  'v': array([   0,    1,    2, ..., 7980, 7981, 7982])},
 {'u': array([   8,    9,   10, ..., 7984, 7985, 7986]),
  'v': array([   0,    1,    2, ..., 7976, 7977, 7978])},
 {'u': array([  11,   12,   13, ..., 7984, 7985, 7986]),
  'v': array([2725, 2726, 2727, ..., 7968, 7969, 7970])},
 {'u': array([  11,   12,   13, ..., 7984, 7985, 7986]),
  'v': array([2709, 2710, 2711, ..., 7952, 7953, 7954])}]

In [33]:
for i,p in enumerate(processed_list):

    print(i)
    torch.save(p,'/home/avt/prediction/Waymo/data_processed/train1/'+ str(i)+'.pt')

/tmp/ipykernel_572560/751831544.py:425: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.ones(len(nbr['u']), np.bool)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [18]:
test1 = torch.load('/home/avt/prediction/Waymo/data_processed/train1/3.pt')

In [21]:
test1['graph'].keys()

dict_keys(['ctrs', 'feats', 'num_nodes', 'node_idcs', 'pre', 'suc', 'pre_pairs', 'suc_pairs', 'left_pairs', 'right_pairs', 'left', 'right', 'letf_boundaries', 'right_boundaries', 'lane_idcs'])

In [27]:
test1['trajs_xyz']

[array([[8485.18261719, 4656.59521484,   42.74338531],
        [8485.18261719, 4656.59521484,   42.7426732 ],
        [8485.18261719, 4656.59521484,   42.7419611 ],
        [8485.18261719, 4656.59521484,   42.74021857],
        [8485.18261719, 4656.59521484,   42.73833565],
        [8485.18261719, 4656.59521484,   42.73615776],
        [8485.18261719, 4656.59521484,   42.73398063],
        [8485.18261719, 4656.59521484,   42.73143543],
        [8485.18261719, 4656.59521484,   42.72860585],
        [8485.18261719, 4656.59521484,   42.72554835],
        [8485.18261719, 4656.59521484,   42.72249077],
        [8485.18261719, 4656.59521484,   42.72134706],
        [8485.18261719, 4656.59521484,   42.72047198],
        [8485.18261719, 4656.59521484,   42.72047198],
        [8485.18261719, 4656.59521484,   42.72047198],
        [8485.18261719, 4656.59521484,   42.71985943],
        [8485.18261719, 4656.59521484,   42.71858425],
        [8485.18261719, 4656.59521484,   42.71635498],
        [8

In [ ]:
[key for key in parsed_list[0].keys()]

In [ ]:
processed_list = Waymo_Motion_Preprocess_3(parsed_list,config)

In [ ]:
z = [False,False]
p = False
if True not in z:
    print('yes')

In [ ]:
A = np.array([ 96,  97,  98])
B = torch.tensor([0])
A[B.numpy()]

In [ ]:
list(A[B.numpy()])